In [51]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pandas as pd
import glob

In [52]:
# Create the Random Forest Regressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)

In [53]:

for train_path in glob.glob('..\data\\train_FD00[0-9].txt'):
    agg_path = train_path.replace('.txt', '_agg.txt')
    rul_path = train_path.replace('train', 'RUL')
    
    train_data = np.genfromtxt(train_path, delimiter=' ')
    agg_data = np.genfromtxt(agg_path, delimiter=' ')
    labels = np.genfromtxt(rul_path)

    X_train = np.hstack((train_data, agg_data))
    
    Y_train = np.array([labels[(int(item[0]) - 1) % len(labels)] for item in X_train])
    
    model.fit(X_train[:, 2:-1], Y_train)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
for test_path in glob.glob('..\data\\test_FD00[0-9].txt'):
    agg_path = test_path.replace('.txt', '_agg.txt')
    rul_path = test_path.replace('test', 'RUL')
    
    test_data = np.genfromtxt(test_path, delimiter=' ')
    agg_data = np.genfromtxt(agg_path, delimiter=' ')
    labels = np.genfromtxt(rul_path)

    X_test = np.hstack((test_data, agg_data))
    
    print(X_test)
    
    
    y_test = np.array([labels[(int(item[0]) - 1) % len(labels)] for item in X_test])
    
    print(y_test)

    
    y_pred = model.predict(X_test[:, 2:-1])

    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print("Mean Squared Error:", mse)
    print("Mean Absolute Error:", mae)
    print("R-squared:", r2)
    
    

[[ 2.3e-03  3.0e-04  1.0e+02 ...  0.0e+00  0.0e+00  5.5e-02]
 [-2.7e-03 -3.0e-04  1.0e+02 ...  0.0e+00  0.0e+00  7.9e-02]
 [ 3.0e-04  1.0e-04  1.0e+02 ...  0.0e+00  0.0e+00  7.1e-02]
 ...
 [-6.0e-04 -3.0e-04  1.0e+02 ...  0.0e+00  0.0e+00  4.3e-02]
 [-3.8e-03  1.0e-04  1.0e+02 ...  0.0e+00  0.0e+00  4.3e-02]
 [ 1.3e-03  3.0e-04  1.0e+02 ...  0.0e+00  0.0e+00  4.3e-02]]
[20. 20. 20. ... 20. 20. 20.]
Mean Squared Error: 36.0
Mean Absolute Error: 6.0
R-squared: 0.0
[[9.99870e+00 2.50200e-01 1.00000e+02 ... 1.70126e+02 7.38300e+00
  4.62400e+00]
 [2.00026e+01 7.00000e-01 1.00000e+02 ... 1.47351e+02 7.38300e+00
  2.00500e+00]
 [3.50045e+01 8.40000e-01 1.00000e+02 ... 1.45500e+02 7.38300e+00
  1.84300e+00]
 ...
 [4.20061e+01 8.40000e-01 1.00000e+02 ... 6.94310e+01 0.00000e+00
  1.10900e+01]
 [2.40000e-03 3.00000e-04 1.00000e+02 ... 6.94310e+01 0.00000e+00
  1.10900e+01]
 [4.20033e+01 8.40000e-01 1.00000e+02 ... 6.94310e+01 0.00000e+00
  1.10900e+01]]
[11. 54. 63. ... 27. 51. 27.]
Mean Square